In [1]:
from load_data import DataGen
# from cnn2rnn import EncoderCNN,DecoderRNN
import argparse
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn import CrossEntropyLoss

import numpy as np 

In [2]:
class EncoderCNN(nn.Module):
    def __init__(self,embeding_size):
        super(EncoderCNN,self).__init__()
        m=models.resnet152(pretrained=True)
        layers=list(m.children())[:-1]
        self.cnn=nn.Sequential(*layers).cuda(2)
        self.linear=nn.Linear(m.fc.in_features,embeding_size).cuda(2)
        self.linear.weight.data.normal_(0.0, 0.02)
        self.linear.bias.data.fill_(0)

    def forward(self,inputs):
        features = self.cnn(inputs)
        features = Variable(features.data)
        features = features.view(features.size(0), -1)
        features = self.linear(features)
        return features

class DecoderRNN(nn.Module):
    def __init__(self,embeding_size,hidden_size,output_size):
        super(DecoderRNN,self).__init__()
        self.embed = nn.Embedding(output_size, embeding_size).cuda(2)
        self.lstm=nn.LSTM(embeding_size,hidden_size,batch_first=True).cuda(2)
        self.linear=nn.Linear(hidden_size,output_size).cuda()
        self.softmax=nn.Softmax(dim=1)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)
        self.embed.weight.data.uniform_(-0.1, 0.1)

    def forward(self,inputs,start_state,lengths):
        print(inputs.size())
        inputs=self.embed(inputs)
        print(inputs.size())
        
        start_state=start_state.unsqueeze(1)
        print(start_state.size())
        inputs=torch.cat((start_state, inputs), 1)
        lengths=lengths.data.numpy().flatten().astype('int').tolist()
        packed = pack_padded_sequence(inputs, lengths, batch_first=True) 
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs

In [3]:
data_gen=DataGen('test_ims.npy','test_labels.npy',batch_size=2)

> Loading Images Done .


In [4]:
epochs=1
learing_rate=0.001
batch_size=256
hidden_size=256
emb_size=256
output_size=18

enc=EncoderCNN(emb_size).float()
dec=DecoderRNN(emb_size,hidden_size,output_size).float()

print('> Models Created ! ')

for e in range(epochs):
    for bi in data_gen.n:
        imb,lbb,lnb=data_gen.get_batch()
        imb=Variable(torch.from_numpy(imb)).float().cuda(2)
        lbb=Variable(torch.from_numpy(lbb)).cuda(2)
        lnb=Variable(torch.from_numpy(lnb)).cuda(2)
        
        print('> forwarding CNN ')
        context=enc.forward(imb)
        print('> CNN forward Done')
        output=dec.forward(lbb,context,lnb)
        print('> Decoder Forward Done')
        break

> Models Created ! 
> forwarding CNN 
> CNN forward Done
torch.Size([2, 6])
torch.Size([2, 6, 256])
torch.Size([2, 1, 256])


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/torch/lib/THC/generic/THCTensorMath.cu:243

In [ ]:
lbb